# TABU SEARCH TSP
    
    

In [1]:
#IMPORTS AND UTILITIES 
import time
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
import copy
import operator
import pandas as pd
import numpy as np

def pseudo_euclidean_distance(point_1, point_2):
    
    x1 = point_1[0]
    y1 = point_1[1]
    x2 = point_2[0]
    y2 = point_2[1]
    
    xd = x1 - x2;
    yd = y1 - y2;
    rij = (((xd*xd + yd*yd)/10.0)**(1/2))
    tij = np.rint(rij);
    
    if (tij<rij):
        dij = tij + 1;
    else:
        dij = tij;
    
    return dij


def cal_route_distance(route_solution): 
        
    Total_distance = 0
    
    new_route_sol = route_solution
    
    for i in range (len(new_route_sol)-1):
        
        x1 = new_route_sol[i][1]
        y1 = new_route_sol[i][2]
        x2 = new_route_sol[i+1][1]
        y2 = new_route_sol[i+1][2]
        
        point_1 = [x1, y1]
        point_2 = [x2, y2]
        
        Total_distance = Total_distance + pseudo_euclidean_distance(point_1, point_2)
        
    return Total_distance

def att48_to_nx3_list(filename):

    with open(filename) as f:
        att48 = f.readlines()

    #Rejects document text 
    pre_size = len(att48)
    att48 =att48[6:pre_size-1] 
    post_size = len(att48)
    
    #separates and splits numbers into list array[cityNUM, xcoord, ycoord]
    #split each line into 3 colums cityID, x_coord, y_coord
    for i in range (post_size):
        att48[i] = att48[i].split()
    
    #Convert all strings in array to ints 
    for i in range (post_size):
        att48[i][0] = int(att48[i][0])
        att48[i][1] = int(att48[i][1])
        att48[i][2] = int(att48[i][2])
         
    return att48


## MAIN

In [5]:
def tabu_search(city_list, num_candidates, iterations, TABU_size_limit):
    
    #establish initial solution and therefore best solution
    intial_solution = city_list
    best_solution = intial_solution
    best_solution_cost = cal_route_distance(best_solution)
    
    #etablish TABU_list
    TABU_list = []
    
    for i in range (iterations):
        
        #create a set of naigbouring solutions as candidates. Best candidate brought forward.
        #A candidate is rejected if in TABU_list and is replaced by a new candidate.
        candidate_list =  calculate_candidate_solutions(best_solution, num_candidates, TABU_list)
    
        best_candidate_cost, best_candidate_sol = fetch_best_candidate(candidate_list)
        
        TABU_list.append(best_candidate_sol)
               
        #At tabu size limit 1 in 1 out.
        if len(TABU_list) >= TABU_size_limit:
            del TABU_list[0]
        
       #check to see if best candidate is a better solution than the current best
        if (best_candidate_cost < best_solution_cost):
            best_solution = best_candidate_sol
            best_solution_cost = best_candidate_cost
        
    return best_solution_cost

def create_naighbour_solution(solution):
    
    #randomly find two cities 
    A = int(random.random()*len(solution))
    B = int(random.random()*len(solution))
    
    city_A = solution[A]
    city_B = solution[B]
    
   # print("A ", A)
   # print("B ", B)
    
    
    new_solution = copy.copy(solution)
    #Swap over
    new_solution[A] = city_B
    new_solution[B] = city_A
    
    return new_solution

def calculate_candidate_solutions(parent_solution, num_candidates, TABU_list):
    
    candidate_list= []
    
    count = 0
    while (count < num_candidates):
       # print("inside count ", count)
        candidate = create_naighbour_solution(parent_solution)
    
        if candidate not in TABU_list:
            candidate_list.append(candidate)
            count = count + 1
        else:
            continue

    return candidate_list


def fetch_best_candidate(candidate_list):
    
    best_candidate = []
    best_candidate_cost = 9999999999999999
    
    for i in range (len(candidate_list)):
        
        candidate_cost = cal_route_distance(candidate_list[i])
        
        if candidate_cost < best_candidate_cost:
            best_candidate_cost = candidate_cost
            best_candidate = candidate_list[i]
            
    return best_candidate_cost, best_candidate
    


# RUN

In [8]:
filename = "att48.tsp" 
initial_route = att48_to_nx3_list(filename) 

city_list = initial_route
num_candidates = 80
iterations = 3000
TABU_size_limit = 100

best_solution_dist = tabu_search(city_list, num_candidates, iterations, TABU_size_limit)
print(best_solution_dist)

15043.0
